In [68]:
import os 
import pandas as pd 
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd
import fastparquet
from sklearn.svm import OneClassSVM
try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")
    
train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')


In [69]:
Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "CodeDecision", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = train['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction","CodeDecision", 'Unnamed: 0'], axis = 1)
Ytest  = test.FlagImpaye

In [71]:
train_100k = Xtrain.sample(n = 100000)
ytrain_100k = Ytrain.sample(n = 100000)
test_100k = Ytest.sample(n = 100000)

In [74]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(train_100k, ytrain_100k)
print(clf.feature_importances_)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(train_100k)
print(X_new.shape)

[0.13182421 0.11123957 0.001532   0.00289059 0.00617631 0.10361692
 0.01879611 0.10973804 0.00353393 0.12014229 0.10318104 0.01071282
 0.00229775 0.00833905 0.00228678 0.00191048 0.11139518 0.0143003
 0.13608664]
(100000, 8)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:402: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [67]:
from xgboost import XGBRFClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import BorderlineSMOTE


nbFoldValid = 5 # nombre de groupes pour la K-CV

models2={'SVC': SVC(),
       'RandomForest': RandomForestClassifier(random_state=0),
       'XGBRFClassifier' : XGBRFClassifier(random_state=0),
       'GradientBoosting' : GradientBoostingClassifier(random_state=0),
        'GradientBoosting' : GradientBoostingClassifier(random_state=0)
       }

In [75]:
# Application des algorithmes 

def applyAlgo(algo,Xtrain, ytrain, Xtest, ytest):

    # On commence par indiquer ce que l'on va faire avec chaque algorithme.
    # on prendra soin de préciser les hyper-paramètres dont dépend l'algorithme

    if algo == "SVC":
        clf = SVC()
        clf.fit(Xtrain, ytrain)
        rankTrain = clf.predict(Xtrain)
        rankTest = clf.predict(Xtest)

    elif algo == "RandomForest":
        clf = RandomForestClassifier(random_state=0)
        clf.fit(Xtrain, ytrain)
        rankTest = clf.predict(Xtest)

    elif algo == "GradientBoostingClassifier":
        clf = GradientBoostingClassifier(random_state=0)
        clf.fit(Xtrain, ytrain)
        rankTest = clf.predict(Xtest)
        
    elif algo == "AdaBoostClassifier":
        clf = AdaBoostClassifier(random_state=0)
        clf.fit(Xtrain, ytrain)
        rankTest = clf.predict(Xtest)
        
    elif algo == "XGBRFClassifier":
        clf = XGBRFClassifier(random_state=0)
        clf.fit(Xtrain, ytrain)
        rankTest = clf.predict(Xtest)

   # Cette deuxième partie permet d'indiquer quelle est la mesure de performance que 
   # vous souhaitez considérer pour votre étude en cours
        ctest = confusion_matrix(ytest, rankTest)
        ftest = round(2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0]),4)

    return (ftest*100)

In [76]:
from sklearn.pipeline import Pipeline
pipe_Sampling = Pipeline([('Smote', BorderlineSMOTE()), ('Modele',model)])
pipe_Sampling = Pipeline([('Smote', BorderlineSMOTE()), ('Modele',model)])

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.metrics import average_precision_score, confusion_matrix
#from functions import loadCsv, oneHotEncodeColumns, data_recovery

In [ ]:
models = ['RandomForest', 'XGBRFClassifier' , 'GradientBoosting', 'GradientBoosting']# 'SVC']

import time
Fmesure = []
execution = []

# Normalisation
normalizer = Normalizer()
normalizer.fit(Xtrain)
Xtrain = normalizer.transform(Xtrain)
Xtest = normalizer.transform(Xtest)

for i in models : 
    start = time.time()
    # Fin normalisation
    apTest = applyAlgo(models, Xtrain, Ytrain, Xtest, Ytest)
    Fmesure.append(apTest) 
    end = time.time()
    elapsed = end - start 
    execution.append(elapsed)